In [313]:
import pandas as pd
import numpy as np

In [314]:
import nltk

In [315]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


False

In [316]:
import re

In [317]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [318]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


def preprocess_text(text):
    # Remove HTML tags (if any)
    text = re.sub(r"<.*?>", "", text)
    
    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)
    
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Remove punctuation
    tokens = [token for token in tokens if token.isalpha()]
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # print(filtered_tokens)
    # # Lemmatization
    # lemmatizer = WordNetLemmatizer()
    # lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Join the tokens back into a single string
    processed_text = " ".join(filtered_tokens)

    
    return processed_text

In [319]:
# amazon=pd.read_csv("Eco_Friendly_Products_Test_Full.xlsx - Sheet1.csv")
amazon=pd.read_csv("Amazon Reviews Validation Data_1.csv")
# amazon=pd.read_csv("amazon_pos.csv")

In [320]:
amazon

,Reviews,Aspect
0,The settings are perfect for all hair types,Adaptability
1,with lots of settings,Adaptability
2,heating element has died,Durability
3,"Also, the diffuser does not stay on at all.",Ease of Use
4,and it won't stay on it at all. Just flys off....,Ease of Use
...,...,...
1176,top no longer stays on.,Performance
1177,unfortunately one of the lids does not fit at all,Performance
1178,"cheap enough that if my husband loses one, we ...",Price
1179,economical price,Price


In [321]:
# a=amazon["Review"]
a=amazon["Reviews"]

In [322]:
q=amazon["Aspect"]

In [323]:
all_values = q.str.split(',').explode()

In [324]:
unique_values = all_values.unique()

In [325]:
len_aspect=len(unique_values)

In [326]:
len_aspect

12

In [327]:
processed_series = a.apply(preprocess_text)

In [328]:
processed_series

0                             settings perfect hair types
1                                           lots settings
2                                    heating element died
3                                      also diffuser stay
4                               wo stay flys see one work
                              ...                        
1176                                     top longer stays
1177                           unfortunately one lids fit
1178                cheap enough husband loses one wo cry
1179                                     economical price
1180    also way seal drinking hole car hit bump stop ...
Name: Reviews, Length: 1181, dtype: object

In [329]:
def extract_nouns(sentence):

    # Tokenize and POS tag the words in the sentence
    tokens = nltk.word_tokenize(sentence)
    parts_of_speech = nltk.pos_tag(tokens)
    x=[word for word, pos in parts_of_speech if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    x=" ".join(x)
    # Extract and return words tagged as NN, NNS, NNP, or NNPS
    return x


In [330]:
nouns = processed_series.apply(extract_nouns)
print(nouns)

0                           settings hair types
1                                 lots settings
2                                       element
3                                 diffuser stay
4                                          work
                         ...                   
1176                                      stays
1177                                   lids fit
1178                                    husband
1179                                      price
1180    way drinking hole car liquid cup holder
Name: Reviews, Length: 1181, dtype: object


In [331]:
pip install -U textblob


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [332]:
from textblob import TextBlob

def extract_nouns_textblob(sentence):
    
    """Extract and return nouns from a given sentence using TextBlob."""
    blob = TextBlob(sentence)
    x=[word for word, tag in blob.tags if tag in ["NN", "NNS", "NNP", "NNPS"]]
    x=" ".join(x)
    return x


In [333]:
nouns_testblob = processed_series.apply(extract_nouns_textblob)
print(nouns)

0                           settings hair types
1                                 lots settings
2                                       element
3                                 diffuser stay
4                                          work
                         ...                   
1176                                      stays
1177                                   lids fit
1178                                    husband
1179                                      price
1180    way drinking hole car liquid cup holder
Name: Reviews, Length: 1181, dtype: object


In [334]:
pip install -U spacy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [335]:
import spacy

In [336]:
nlpt = spacy.load("en_core_web_sm")

In [337]:
def extract_nouns_spacy(sentence):
    """Extract and return nouns from a given sentence using spaCy."""
    doc = nlpt(sentence)
    x=[token.text for token in doc if token.pos_ in ["NOUN"]]
    x=" ".join(x)
    return x

In [338]:
doc = processed_series.apply(extract_nouns_spacy)
doc

0                                     settings hair types
1                                           lots settings
2                                         heating element
3                                                diffuser
4                                               flys work
                              ...                        
1176                                                  top
1177                                             lids fit
1178                                              husband
1179                                                price
1180    way seal drinking hole car bump stop liquid cu...
Name: Reviews, Length: 1181, dtype: object

In [339]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [340]:
# Step 2: Convert text to numerical vectors using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(processed_series).toarray()

In [341]:
# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the documents and transform the documents into TF-IDF vectors
tfidf_vectors = vectorizer.fit_transform(processed_series).toarray()

In [342]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

In [343]:
num_clusters = 12

In [344]:
nltk = vectorizer.fit_transform(nouns).toarray()

In [345]:
spacy_=vectorizer.fit_transform(doc).toarray()

In [346]:
TextBlob_=vectorizer.fit_transform(nouns_testblob).toarray()

# Cluster for 12 aspects

In [347]:
num_clusters = 12

In [348]:
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
word_vectors = normalize(X) 
centroids = word_vectors[np.random.choice(word_vectors.shape[0], num_clusters, replace=False), :]


kmeans = KMeans(n_clusters=num_clusters, random_state=0)
labels = kmeans.fit_predict(word_vectors)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [349]:
# for nltk
nltk = normalize(nltk)
centroids_nltk = nltk[np.random.choice(nltk.shape[0], num_clusters, replace=False), :]
kmeans_nltk = KMeans(n_clusters=num_clusters, random_state=0)
labels_nltk = kmeans.fit_predict(nltk)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [350]:
# for spacy
spacy_ = normalize(spacy_)
centroids_spacy = spacy_[np.random.choice(spacy_.shape[0], num_clusters, replace=False), :]
kmeans_spacy = KMeans(n_clusters=num_clusters, random_state=0)
labels_spacy = kmeans.fit_predict(spacy_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [351]:
# for TextBlob
TextBlob_ = normalize(TextBlob_)
centroids_TextBlob = TextBlob_[np.random.choice(TextBlob_.shape[0], num_clusters, replace=False), :]
kmeans_TextBlob = KMeans(n_clusters=num_clusters, random_state=0)
labels_TextBlob = kmeans.fit_predict(TextBlob_)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [352]:
# Create DataFrame
df_spacy = pd.DataFrame({
    'Sentence': doc,
    'Cluster': labels_spacy
})

# Show the DataFrame
print(df_spacy)

                                               Sentence  Cluster
0                                   settings hair types        0
1                                         lots settings        0
2                                       heating element        0
3                                              diffuser        0
4                                             flys work        0
...                                                 ...      ...
1176                                                top        0
1177                                           lids fit        0
1178                                            husband        0
1179                                              price        7
1180  way seal drinking hole car bump stop liquid cu...        0

[1181 rows x 2 columns]


In [209]:
sentences_testblob=np.array(nouns_testblob)
sentences_spacy=np.array(doc)
sentences_nltk=np.array(nouns)


In [353]:
# Create DataFrame
df = pd.DataFrame({
    'Sentence': processed_series,
    'Cluster': labels
})

# Show the DataFrame
print(df)

                                               Sentence  Cluster
0                           settings perfect hair types        8
1                                         lots settings        8
2                                  heating element died        8
3                                    also diffuser stay        8
4                             wo stay flys see one work        8
...                                                 ...      ...
1176                                   top longer stays        8
1177                         unfortunately one lids fit        8
1178              cheap enough husband loses one wo cry        8
1179                                   economical price        8
1180  also way seal drinking hole car hit bump stop ...        8

[1181 rows x 2 columns]


In [354]:
# Create DataFrame
df_textblob = pd.DataFrame({
    'Sentence': sentences_testblob,
    'Cluster': labels_TextBlob
})

# Show the DataFrame
print(df_textblob)

                                     Sentence  Cluster
0                         settings hair types        1
1                               lots settings        1
2                                     element        1
3                               diffuser stay        1
4                                        work        9
...                                       ...      ...
1176                                    stays        1
1177                                 lids fit        1
1178                                  husband        1
1179                                    price       10
1180  way drinking hole car liquid cup holder        1

[1181 rows x 2 columns]


In [355]:
# Create DataFrame
df_nltk = pd.DataFrame({
    'Sentence': sentences_nltk,
    'Cluster': labels_nltk
})

# Show the DataFrame
print(df_nltk)

                                     Sentence  Cluster
0                         settings hair types        1
1                               lots settings        1
2                                     element        1
3                               diffuser stay        1
4                                        work        9
...                                       ...      ...
1176                                    stays        1
1177                                 lids fit        1
1178                                  husband        1
1179                                    price       10
1180  way drinking hole car liquid cup holder        1

[1181 rows x 2 columns]


In [213]:
# import numpy as np
# from sklearn.mixture import GaussianMixture
# import matplotlib.pyplot as plt

# # Assuming glove_vector and word_vectors are NumPy arrays
# # Replace these with your actual data
# glove_vector = tfidf_vectors

# # Function to fit the model and plot results
# def fit_and_plot(data, title):
#     best_gmm = None
#     best_score = float('-inf')
#     for n_components in range(3, 25):
#         gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=0)
#         gmm.fit(data)
#         score = gmm.bic(data)
#         if score > best_score:
#             best_score = score
#             best_gmm = gmm

#     print(f'Best GMM for {title}: {best_gmm}')
#     print(f'Summary: Means - {best_gmm.means_}, Covariances - {best_gmm.covariances_}')

#     plt.figure()
#     plt.scatter(data[:, 0], data[:, 1], c=best_gmm.predict(data))
#     plt.title(title)
#     plt.show()

# # Fit and plot for glove_vector and word_vectors
# fit_and_plot(glove_vector, 'Glove Vectors')



In [214]:
# import numpy as np
# from sklearn.mixture import GaussianMixture
# import matplotlib.pyplot as plt

# # Assuming glove_vector and word_vectors are NumPy arrays
# # Replace these with your actual data
# glove_vector = X

# # Function to fit the model and plot results
# def fit_and_plot(data, title):
#     best_gmm = None
#     best_score = float('-inf')
#     for n_components in range(3, 13):
#         gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=0)
#         gmm.fit(data)
#         score = gmm.bic(data)
#         if score > best_score:
#             best_score = score
#             best_gmm = gmm

#     print(f'Best GMM for {title}: {best_gmm}')
#     print(f'Summary: Means - {best_gmm.means_}, Covariances - {best_gmm.covariances_}')

#     plt.figure()
#     plt.scatter(data[:, 0], data[:, 1], c=best_gmm.predict(data))
#     plt.title(title)
#     plt.show()

# # Fit and plot for glove_vector and word_vectors
# fit_and_plot(glove_vector, 'Glove Vectors')



In [215]:
# import numpy as np
# from sklearn.mixture import GaussianMixture
# import matplotlib.pyplot as plt

# # Assuming glove_vector and word_vectors are NumPy arrays
# # Replace these with your actual data
# glove_vector = tfidf_vectors

# # Function to fit the model and plot results
# def fit_and_plot(data, title):
#     best_gmm = None
#     best_score = float('-inf')
#     for n_components in range(3, 7):
#         gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=0)
#         gmm.fit(data)
#         score = gmm.bic(data)
#         if score > best_score:
#             best_score = score
#             best_gmm = gmm

#     print(f'Best GMM for {title}: {best_gmm}')
#     print(f'Summary: Means - {best_gmm.means_}, Covariances - {best_gmm.covariances_}')

#     plt.figure()
#     plt.scatter(data[:, 0], data[:, 1], c=best_gmm.predict(data))
#     plt.title(title)
#     plt.show()

# # Fit and plot for glove_vector and word_vectors
# fit_and_plot(glove_vector, 'Glove Vectors')



# ploting

In [216]:
import plotly.express as px
import plotly.io as pio
# pio.renderers.default = "browser"

In [217]:
from sklearn.manifold import TSNE

perplexity_value = min(30, len(word_vectors) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(word_vectors)

In [218]:
import plotly.graph_objects as go
import plotly.io as pio

fig = go.Figure()

for i in range(num_clusters):
    indices = labels == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE (normal)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [219]:
import plotly.graph_objects as go

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(word_vectors) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(word_vectors)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(normal)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## textblobl

In [220]:
perplexity_value = min(30, len(TextBlob_) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(TextBlob_)

In [221]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_TextBlob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(TextBlob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [222]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(TextBlob_) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(TextBlob_)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_TextBlob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(TextBlob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [223]:
perplexity_value = min(30, len(spacy_) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(spacy_)

In [224]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [225]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(spacy_) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(spacy_)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [226]:
perplexity_value = min(30, len(nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(nltk)

In [227]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(nltk)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [228]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(nltk)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# kmeans with 6 aspects

In [377]:
num_clusters = 6 # Example number of clusters

In [378]:

kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(word_vectors)
labels_6 = kmeans.fit_predict(word_vectors)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [231]:
kmeans_textblob = KMeans(n_clusters=num_clusters)
kmeans_textblob.fit(TextBlob_)
labels_textblob_6 = kmeans_textblob.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [232]:
kmeans_spacy = KMeans(n_clusters=num_clusters)
kmeans_spacy.fit(spacy_)
labels_spacy_6 = kmeans_spacy.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [233]:
kmeans_nltk = KMeans(n_clusters=num_clusters)
kmeans_nltk.fit(nltk)
labels_nltk_6 = kmeans_nltk.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



# df

In [234]:
# Create DataFrame
df_spacy_6 = pd.DataFrame({
    'Sentence': doc,
    'Cluster': labels_spacy_6
})

# Show the DataFrame
print(df_spacy_6)

                                               Sentence  Cluster
0                                   settings hair types        1
1                                         lots settings        1
2                                       heating element        1
3                                              diffuser        1
4                                             flys work        1
...                                                 ...      ...
1176                                                top        1
1177                                           lids fit        1
1178                                            husband        1
1179                                              price        4
1180  way seal drinking hole car bump stop liquid cu...        1

[1181 rows x 2 columns]


In [379]:
# Create DataFrame
df_6 = pd.DataFrame({
    'Sentence': processed_series,
    'Cluster': labels_6
})

# Show the DataFrame
print(df_6)

                                               Sentence  Cluster
0                           settings perfect hair types        4
1                                         lots settings        4
2                                  heating element died        4
3                                    also diffuser stay        4
4                             wo stay flys see one work        4
...                                                 ...      ...
1176                                   top longer stays        4
1177                         unfortunately one lids fit        4
1178              cheap enough husband loses one wo cry        4
1179                                   economical price        4
1180  also way seal drinking hole car hit bump stop ...        4

[1181 rows x 2 columns]


In [236]:
# Create DataFrame
df_textblob_6 = pd.DataFrame({
    'Sentence': sentences_testblob,
    'Cluster': labels_textblob_6
})

# Show the DataFrame
print(df_textblob_6)

                                     Sentence  Cluster
0                         settings hair types        5
1                               lots settings        5
2                                     element        5
3                               diffuser stay        5
4                                        work        5
...                                       ...      ...
1176                                    stays        5
1177                                 lids fit        5
1178                                  husband        5
1179                                    price        3
1180  way drinking hole car liquid cup holder        5

[1181 rows x 2 columns]


In [237]:
# Create DataFrame
df_nltk_6 = pd.DataFrame({
    'Sentence': sentences_nltk,
    'Cluster': labels_nltk_6
})

# Show the DataFrame
print(df_nltk_6)

                                     Sentence  Cluster
0                         settings hair types        4
1                               lots settings        4
2                                     element        4
3                               diffuser stay        4
4                                        work        4
...                                       ...      ...
1176                                    stays        4
1177                                 lids fit        4
1178                                  husband        4
1179                                    price        4
1180  way drinking hole car liquid cup holder        4

[1181 rows x 2 columns]


# ploting

In [238]:
perplexity_value = min(30, len(word_vectors) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(word_vectors)

In [239]:

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_6 == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(normal)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [240]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(word_vectors) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(word_vectors)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_6 == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(normal)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## textblobl

In [241]:
perplexity_value = min(30, len(TextBlob_) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(TextBlob_)

In [242]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob_6 == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(TextBlob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [243]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(TextBlob_) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(TextBlob_)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob_6 == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(TextBlob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [244]:
perplexity_value = min(30, len(spacy_) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(spacy_)

In [245]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy_6 == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [246]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(spacy_) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(spacy_)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy_6 == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [247]:
perplexity_value = min(30, len(nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(nltk)

In [248]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk_6 == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(nltk)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [249]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk_6 == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(nltk)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# GMM

In [250]:
from sklearn.mixture import GaussianMixture

In [404]:
gmm = GaussianMixture(n_components = 12)

In [405]:
gmm_6 = GaussianMixture(n_components = 6)

In [406]:
gmm.fit(word_vectors)

GaussianMixture(n_components=12)

In [407]:
labels_gmm = gmm.predict(word_vectors)

In [255]:
gmm.fit(TextBlob_)
labels_textblob_gmm = gmm.predict(TextBlob_)

In [256]:
gmm.fit(spacy_)
labels_spacy_gmm = gmm.predict(spacy_)

In [257]:
gmm.fit(nltk)
labels_nltk_gmm = gmm.predict(nltk)

# df

In [258]:
# Create DataFrame
df_spacy_gmm = pd.DataFrame({
    'Sentence': doc,
    'Cluster': labels_spacy_gmm
})

# Show the DataFrame
print(df_spacy)

                                               Sentence  Cluster
0                                   settings hair types        0
1                                         lots settings        0
2                                       heating element        0
3                                              diffuser        0
4                                             flys work        0
...                                                 ...      ...
1176                                                top        0
1177                                           lids fit        0
1178                                            husband        0
1179                                              price        7
1180  way seal drinking hole car bump stop liquid cu...        0

[1181 rows x 2 columns]


In [409]:
# Create DataFrame
df_gmm = pd.DataFrame({
    'Sentence': processed_series,
    'Cluster': labels_gmm
})

# Show the DataFrame
print(df_gmm)

                                               Sentence  Cluster
0                           settings perfect hair types        4
1                                         lots settings        4
2                                  heating element died        4
3                                    also diffuser stay       11
4                             wo stay flys see one work       11
...                                                 ...      ...
1176                                   top longer stays        4
1177                         unfortunately one lids fit       11
1178              cheap enough husband loses one wo cry       11
1179                                   economical price        4
1180  also way seal drinking hole car hit bump stop ...        4

[1181 rows x 2 columns]


In [260]:
# Create DataFrame
df_textblob_gmm = pd.DataFrame({
    'Sentence': sentences_testblob,
    'Cluster': labels_textblob_gmm
})

# Show the DataFrame
print(df_textblob)

                                     Sentence  Cluster
0                         settings hair types        1
1                               lots settings        1
2                                     element        1
3                               diffuser stay        1
4                                        work        9
...                                       ...      ...
1176                                    stays        1
1177                                 lids fit        1
1178                                  husband        1
1179                                    price       10
1180  way drinking hole car liquid cup holder        1

[1181 rows x 2 columns]


In [261]:
# Create DataFrame
df_nltk_gmm = pd.DataFrame({
    'Sentence': sentences_nltk,
    'Cluster': labels_nltk_gmm
})

# Show the DataFrame
print(df_nltk)

                                     Sentence  Cluster
0                         settings hair types        1
1                               lots settings        1
2                                     element        1
3                               diffuser stay        1
4                                        work        9
...                                       ...      ...
1176                                    stays        1
1177                                 lids fit        1
1178                                  husband        1
1179                                    price       10
1180  way drinking hole car liquid cup holder        1

[1181 rows x 2 columns]


# ploting

In [262]:
num_clusters=12

In [263]:


perplexity_value = min(30, len(word_vectors) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(word_vectors)

In [264]:

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_gmm == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(normal)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [265]:
import plotly.graph_objects as go

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(word_vectors) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(word_vectors)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_gmm == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(normal)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## textblobl

In [266]:
perplexity_value = min(30, len(TextBlob_) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(TextBlob_)

In [267]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob_gmm == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(TextBlob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [268]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(TextBlob_) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(TextBlob_)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob_gmm == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(TextBlob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [269]:
perplexity_value = min(30, len(spacy_) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(spacy_)

In [270]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy_gmm == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [271]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(spacy_) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(spacy_)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy_gmm == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [272]:
perplexity_value = min(30, len(nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(nltk)

In [273]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk_gmm == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(nltk)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [274]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk_gmm == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Cluster(nltk)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# 6 cluster

In [275]:
gmm_6.fit(nltk)
labels_nltk_gmm_6 = gmm_6.predict(nltk)

In [410]:
gmm_6.fit(word_vectors)
labels_gmm_6 = gmm_6.predict(word_vectors)

In [277]:
gmm_6.fit(spacy_)
labels_spacy_gmm_6 = gmm_6.predict(spacy_)

In [278]:
gmm_6.fit(TextBlob_ )
labels_textblob_gmm_6 = gmm_6.predict(TextBlob_)

In [279]:
num_clusters=6

# ploting

In [280]:


perplexity_value = min(30, len(word_vectors) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(word_vectors)

In [281]:

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_gmm_6 == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(normal)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [282]:
import plotly.graph_objects as go

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(word_vectors) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(word_vectors)
# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_gmm_6 == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(normal)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## textblobl

In [283]:
perplexity_value = min(30, len(TextBlob_) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(TextBlob_)

In [284]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob_gmm_6 == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(TextBlob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [285]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(TextBlob_) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(TextBlob_)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob_gmm_6 == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(TextBlob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [286]:
perplexity_value = min(30, len(spacy_) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(spacy_)

In [287]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy_gmm_6 == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [288]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(spacy_) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(spacy_)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy_gmm_6 == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [289]:
perplexity_value = min(30, len(nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(nltk)

In [290]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk_gmm_6 == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(nltk)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [291]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk_gmm_6 == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(nltk)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# df


In [292]:
# Create DataFrame
df_spacy_gmm_6 = pd.DataFrame({
    'Sentence': doc,
    'Cluster': labels_spacy_gmm_6
})

# Show the DataFrame
print(df_spacy)

                                               Sentence  Cluster
0                                   settings hair types        0
1                                         lots settings        0
2                                       heating element        0
3                                              diffuser        0
4                                             flys work        0
...                                                 ...      ...
1176                                                top        0
1177                                           lids fit        0
1178                                            husband        0
1179                                              price        7
1180  way seal drinking hole car bump stop liquid cu...        0

[1181 rows x 2 columns]


In [412]:
# Create DataFrame
df_gmm_6 = pd.DataFrame({
    'Sentence': processed_series,
    'Cluster': labels_gmm_6
})

# Show the DataFrame
print(df_gmm_6)

                                               Sentence  Cluster
0                           settings perfect hair types        0
1                                         lots settings        4
2                                  heating element died        4
3                                    also diffuser stay        4
4                             wo stay flys see one work        4
...                                                 ...      ...
1176                                   top longer stays        4
1177                         unfortunately one lids fit        4
1178              cheap enough husband loses one wo cry        4
1179                                   economical price        4
1180  also way seal drinking hole car hit bump stop ...        4

[1181 rows x 2 columns]


In [294]:
# Create DataFrame
df_textblob_gmm_6 = pd.DataFrame({
    'Sentence': sentences_testblob,
    'Cluster': labels_textblob_gmm_6
})

# Show the DataFrame
print(df_textblob)

                                     Sentence  Cluster
0                         settings hair types        1
1                               lots settings        1
2                                     element        1
3                               diffuser stay        1
4                                        work        9
...                                       ...      ...
1176                                    stays        1
1177                                 lids fit        1
1178                                  husband        1
1179                                    price       10
1180  way drinking hole car liquid cup holder        1

[1181 rows x 2 columns]


In [295]:
# Create DataFrame
df_nltk_gmm_6 = pd.DataFrame({
    'Sentence': sentences_nltk,
    'Cluster': labels_nltk_gmm_6
})

# Show the DataFrame
print(df_nltk)

                                     Sentence  Cluster
0                         settings hair types        1
1                               lots settings        1
2                                     element        1
3                               diffuser stay        1
4                                        work        9
...                                       ...      ...
1176                                    stays        1
1177                                 lids fit        1
1178                                  husband        1
1179                                    price       10
1180  way drinking hole car liquid cup holder        1

[1181 rows x 2 columns]


In [296]:
type(q)

pandas.core.series.Series

In [403]:
df_gmm

,Sentence,Cluster
0,settings perfect hair types,8
1,lots settings,8
2,heating element died,8
3,also diffuser stay,6
4,wo stay flys see one work,8
...,...,...
1176,top longer stays,8
1177,unfortunately one lids fit,8
1178,cheap enough husband loses one wo cry,8
1179,economical price,8


In [413]:
# add the aspect as per the sentenecs from amazon df to  df_nltk
df_=df_gmm.join(q)

In [415]:
df_

,Sentence,Cluster,Aspect
0,settings perfect hair types,4,Adaptability
1,lots settings,4,Adaptability
2,heating element died,4,Durability
3,also diffuser stay,11,Ease of Use
4,wo stay flys see one work,11,Ease of Use
...,...,...,...
1176,top longer stays,4,Performance
1177,unfortunately one lids fit,11,Performance
1178,cheap enough husband loses one wo cry,11,Price
1179,economical price,4,Price


In [416]:
df_.Cluster.value_counts()

Cluster
4     706
1      87
7      77
3      68
2      62
11     47
10     41
8      34
5      28
9      20
0      10
6       1
Name: count, dtype: int64

In [417]:
clusters_=set(labels)

In [418]:
cluster_=list(clusters_)

In [419]:
print(type(cluster_))

<class 'list'>


In [420]:
cluster_

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [421]:
feature_cluster_table = pd.DataFrame(index=unique_values, columns=cluster_)

In [422]:
aa=df_['Cluster']
aa=aa.explode().to_list()
print(set(aa))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


In [424]:
for value in unique_values:
    for cluster in labels_nltk:
        # Count occurrences of the value in this cluster
        count = df_[(df_['Aspect'] == value) & (df_['Cluster'] == cluster)].shape[0]
        feature_cluster_table.at[value, cluster] = count

# Display the table
print(feature_cluster_table)

                     0   1   2   3    4   5  6   7   8   9   10  11
Adaptability          1   1   2   7   54   0  0   6   0   1   3   4
Durability            0   0  19   7  153   0  0  12   1  15   2   8
Ease of Use           1  48   1   0   44   0  0   1   0   0   0   5
Ergonomics            1   1   1   0   29  27  0  12   1   0   2   0
Interference          3   0   3   1   53   0  0   8   1   3   1   7
Performance           0   1  25  32  125   1  0   8  25   1  14  13
Use Efficiency        1   2   0   0   44   0  0   0   0   0   0   3
Aesthetics            1   4   1   8   95   0  1  25   1   0   9   1
Ease of Reprocessing  0  29   7   1   53   0  0   3   0   0   0   1
Ease of Storage       2   1   0   1    7   0  0   0   0   0   0   1
Price                 0   0   3  11   40   0  0   2   5   0  10   3
Safety                0   0   0   0    9   0  0   0   0   0   0   1


In [425]:
df_gmm_6

,Sentence,Cluster
0,settings perfect hair types,0
1,lots settings,4
2,heating element died,4
3,also diffuser stay,4
4,wo stay flys see one work,4
...,...,...
1176,top longer stays,4
1177,unfortunately one lids fit,4
1178,cheap enough husband loses one wo cry,4
1179,economical price,4


In [426]:
# add the aspect as per the sentenecs from amazon df to  df_nltk
df_=df_gmm_6.join(q)

In [427]:
clusters_6=set(labels_6)

In [428]:
clusters_6

{0, 1, 2, 3, 4, 5}

In [429]:
clusters_6=list(clusters_6)

In [430]:
clusters_6

[0, 1, 2, 3, 4, 5]

In [431]:
feature_cluster_table_gmm_6 = pd.DataFrame(index=unique_values, columns=clusters_6)

In [432]:
feature_cluster_table_gmm_6

,0,1,2,3,4,5
Adaptability,NaN,NaN,NaN,NaN,NaN,NaN
Durability,NaN,NaN,NaN,NaN,NaN,NaN
Ease of Use,NaN,NaN,NaN,NaN,NaN,NaN
Ergonomics,NaN,NaN,NaN,NaN,NaN,NaN
Interference,NaN,NaN,NaN,NaN,NaN,NaN
Performance,NaN,NaN,NaN,NaN,NaN,NaN
Use Efficiency,NaN,NaN,NaN,NaN,NaN,NaN
Aesthetics,NaN,NaN,NaN,NaN,NaN,NaN
Ease of Reprocessing,NaN,NaN,NaN,NaN,NaN,NaN
Ease of Storage,NaN,NaN,NaN,NaN,NaN,NaN


In [433]:
for value in unique_values:
    for cluster in labels_6:
        # Count occurrences of the value in this cluster
        count = df_[(df_['Aspect'] == value) & (df_['Cluster'] == cluster)].shape[0]
        feature_cluster_table_gmm_6.at[value, cluster] = count

# Display the table
print(feature_cluster_table_gmm_6)

                      0   1   2   3    4   5
Adaptability          1   3   9   0   65   1
Durability            9   2   6  29  164   7
Ease of Use           1   0   1   0   98   0
Ergonomics            1   3   0   0   56  14
Interference          3   1   1   0   72   3
Performance           4  24  36   1  177   3
Use Efficiency        6   0   0   0   44   0
Aesthetics            1  10   9   0  110  16
Ease of Reprocessing  3   0   1   0   89   1
Ease of Storage       0   0   1   0   11   0
Price                 1  12  11   0   50   0
Safety                0   0   0   0   10   0
